In [38]:
from copy import deepcopy
import json

slot_meta = json.load(open("/opt/ml/input/data/train_dataset/slot_meta.json"))
ontology_file = json.load(open("/opt/ml/input/data/train_dataset/ontology.json"))
categorical_slots = [s for s in slot_meta if len(ontology_file[s]) <= 10]

direction_types = [s for s in categorical_slots if s.endswith('지역')]
date_types = [s for s in categorical_slots if s.endswith('요일')]

def process_value(string):
    domain, slot, value = string.split("-")
    slot_type = "-".join((domain, slot))

    string = string.replace(' & ', '&')
    string = string.replace(' = ', '=')

    string = string.replace('분식 회개', '분식회개')
    string = string.replace('회ne', '분식회개')
    string = string.replace('-회개', '-분식회개')
    
    string = string.replace(" ( ", " (")
    string = string.replace(" ) )", ")")
    string = string.replace(" )", ")")

    if 'do' in string:
        if slot_type in direction_types:
            string = string.replace('do ', '서울 ')
        else:
            string = "-".join((slot_type, 'dontcare'))
    
    if slot_type in date_types:
        if value not in ontology_file[slot_type]:
            string = ""

    if '식당' in string and '랑또' in string :
        string = '식당-이름-사케랑또'
    return string

def process_predictions_csv(submission_csv_dir):
    predictions = open(submission_csv_dir, 'r')
    predictions = json.load(predictions)
    new_predictions = deepcopy(predictions)
    for k,v in predictions.items():
        new_vals = []
        for values in v:
            new_val = process_value(values)
            if new_val:
                new_vals.append(new_val)
        new_predictions[k] = new_vals
    return new_predictions

# ##file_name 바꾸셈!! (바꾸고 싶은 submission csv 파일의 경로이름)

In [35]:
################################################################
file_path = '/opt/ml/code/prediction/kfold'
file_name = 'kfold-pseudo-soft-ensemble.csv'
################################################################

new_predictions = process_predictions_csv(os.path.join(file_path, file_name))

# ##새로 저장할 csv파일의 이름 정하세요!!! (new_file_name)

In [37]:
import os
new_folder = '/opt/ml/new_processed_submission'
os.makedirs(new_folder,exist_ok=True)
#########################################################
new_file_name = f"my-processed_kfold-pseudo-soft-ensemble.csv"
#########################################################

new_saved_file_name = f'{new_folder}/{new_file_name}'
json.dump(new_predictions, open(new_saved_file_name, 'w'), indent=2, ensure_ascii=False) 

In [21]:
slot_meta = json.load(open("/opt/ml/input/data/train_dataset/slot_meta.json"))
ontology_file = json.load(open("/opt/ml/input/data/train_dataset/ontology.json"))
def check_categorical(did, categorical_slots, slot_type, value):
    if slot_type in categorical_slots:
        if value not in ontology_file[slot_type]:
            print(did, slot_type, value)
    

In [8]:
slot_meta = json.load(open("/opt/ml/input/data/train_dataset/slot_meta.json"))
ontology_file = json.load(open("/opt/ml/input/data/train_dataset/ontology.json"))

In [9]:
categorical_slots = []
for s in slot_meta:
    if len(ontology_file[s]) < 15:
        categorical_slots.append(s)


In [10]:
categorical_slots

['관광-경치 좋은',
 '관광-교육적',
 '관광-도보 가능',
 '관광-문화 예술',
 '관광-역사적',
 '관광-종류',
 '관광-주차 가능',
 '관광-지역',
 '숙소-가격대',
 '숙소-도보 가능',
 '숙소-수영장 유무',
 '숙소-스파 유무',
 '숙소-예약 기간',
 '숙소-예약 명수',
 '숙소-예약 요일',
 '숙소-인터넷 가능',
 '숙소-조식 가능',
 '숙소-종류',
 '숙소-주차 가능',
 '숙소-지역',
 '숙소-헬스장 유무',
 '숙소-흡연 가능',
 '식당-가격대',
 '식당-도보 가능',
 '식당-야외석 유무',
 '식당-예약 명수',
 '식당-예약 요일',
 '식당-인터넷 가능',
 '식당-종류',
 '식당-주류 판매',
 '식당-주차 가능',
 '식당-지역',
 '식당-흡연 가능',
 '지하철-출발 시간',
 '택시-종류']

In [24]:
for slot in categorical_slots:
    if slot.endswith('지역'):
        print(slot)

관광-지역
숙소-지역
식당-지역


In [23]:
ontology_file['지하철-출발 시간']

['none',
 'dontcare',
 '13:30',
 '13:10',
 '12:40',
 '12:30',
 '12:00',
 '12:20',
 '12:50',
 '13:40',
 '12:10',
 '13:20']

In [22]:
from copy import deepcopy
import json

predictions = json.load(open(os.path.join(file_path, file_name)))
new_predictions = deepcopy(predictions)
for k,v in predictions.items():
    new_value = []
    for values in v:
        splited_value = values.split('-')
        check_categorical(k, categorical_slots, "-".join((splited_value[0], splited_value[1])), splited_value[2])
        new_value.append('-'.join(splited_value))
    new_predictions[k] = new_value

orange-dream-2065:관광_숙소_식당_11-13 관광-종류 dont
late-band-0535:식당_관광_숙소_9-3 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-4 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-5 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-6 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-7 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-8 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-9 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-10 식당-예약 요일 6
blue-firefly-5738:식당_관광_택시_12-2 식당-예약 요일 09
blue-firefly-5738:식당_관광_택시_12-4 식당-예약 요일 09
blue-firefly-5738:식당_관광_택시_12-5 식당-예약 요일 09
royal-grass-5366:식당_관광_택시_13-1 식당-종류 dont
gentle-dream-4724:식당_관광_지하철_12-3 식당-예약 요일 06
gentle-dream-4724:식당_관광_지하철_12-4 식당-예약 요일 06
gentle-dream-4724:식당_관광_지하철_12-5 식당-예약 요일 06
gentle-dream-4724:식당_관광_지하철_12-6 식당-예약 요일 06
gentle-dream-4724:식당_관광_지하철_12-7 식당-예약 요일 06
sparkling-hall-3405:숙소_식당_관광_13-6 식당-예약 요일 9
sparkling-hall-3405:숙소_식당_관광_13-7 식당-예약 요일 9
sparkling-hall-3405:숙소_식당_관광_13-8 식당-예약 요일 9
solitary-cloud-8904:식당_숙소_택시_13-6 숙소-지역 do 서쪽
solitary-cloud-8904:식당_숙소_택시_13-7 숙소-지역 do 서쪽
solitary-cloud-89

SyntaxError: 'return' outside function (<ipython-input-22-83eb94008761>, line 13)

In [40]:
from copy import deepcopy
import json

new_folder = '/opt/ml/new_processed_submission'
new_file_name = f"processed_kfold-pseudo-soft-ensemble.csv"

predictions = json.load(open(os.path.join(new_folder, new_file_name)))
new_predictions = deepcopy(predictions)
for k,v in predictions.items():
    new_value = []
    for values in v:
        splited_value = values.split('-')
        check_categorical(k, categorical_slots, "-".join((splited_value[0], splited_value[1])), splited_value[2])
        new_value.append('-'.join(splited_value))
    new_predictions[k] = new_value

late-band-0535:식당_관광_숙소_9-3 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-4 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-5 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-6 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-7 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-8 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-9 식당-예약 요일 6
late-band-0535:식당_관광_숙소_9-10 식당-예약 요일 6
blue-firefly-5738:식당_관광_택시_12-2 식당-예약 요일 09
blue-firefly-5738:식당_관광_택시_12-4 식당-예약 요일 09
blue-firefly-5738:식당_관광_택시_12-5 식당-예약 요일 09
royal-grass-5366:식당_관광_택시_13-1 식당-종류 dont
gentle-dream-4724:식당_관광_지하철_12-3 식당-예약 요일 06
gentle-dream-4724:식당_관광_지하철_12-4 식당-예약 요일 06
gentle-dream-4724:식당_관광_지하철_12-5 식당-예약 요일 06
gentle-dream-4724:식당_관광_지하철_12-6 식당-예약 요일 06
gentle-dream-4724:식당_관광_지하철_12-7 식당-예약 요일 06
sparkling-hall-3405:숙소_식당_관광_13-6 식당-예약 요일 9
sparkling-hall-3405:숙소_식당_관광_13-7 식당-예약 요일 9
sparkling-hall-3405:숙소_식당_관광_13-8 식당-예약 요일 9
solitary-cloud-8904:식당_숙소_택시_13-6 숙소-지역 서울서쪽
solitary-cloud-8904:식당_숙소_택시_13-7 숙소-지역 서울서쪽
solitary-cloud-8904:식당_숙소_택시_13-8 숙소-지역 서울서쪽
small-cake-0338:숙소

In [39]:
from copy import deepcopy
import json

new_folder = '/opt/ml/new_processed_submission'
os.makedirs(new_folder,exist_ok=True)
new_file_name = f"my-processed_kfold-pseudo-soft-ensemble.csv"

predictions = json.load(open(os.path.join(new_folder, new_file_name)))
new_predictions = deepcopy(predictions)
for k,v in predictions.items():
    new_value = []
    for values in v:
        splited_value = values.split('-')
        check_categorical(k, categorical_slots, "-".join((splited_value[0], splited_value[1])), splited_value[2])
        new_value.append('-'.join(splited_value))
    new_predictions[k] = new_value

small-cake-0338:숙소_관광_택시_13-9 관광-지역 서울 중앙care
weathered-breeze-0698:숙소_식당_관광_12-1 숙소-종류 에버이네집
weathered-breeze-0698:숙소_식당_관광_12-2 숙소-종류 에버비엔비
throbbing-butterfly-3297:숙소_식당_관광_13-6 숙소-지역 서울 북
silent-paper-9948:숙소_식당_택시_11-7 식당-종류 베어집
